In [1]:
import numpy as np
from tensorflow import keras
import utils

In [2]:
history_steps = 50
forecast_steps = 10
(x_train, y_train), (x_val, y_val), (x_test, y_test) = \
    utils.load_time_series_data_multiple_forecast(history_steps, forecast_steps)
print(f"x_train.shape = {x_train.shape}, y_train.shape = {y_train.shape}")
print(f"x_val.shape = {x_val.shape}, y_val.shape = {y_val.shape}")
print(f"x_test.shape = {x_test.shape}, y_test.shape = {y_test.shape}")

x_train.shape = (7000, 50, 1), y_train.shape = (7000, 50, 10)
x_val.shape = (2000, 50, 1), y_val.shape = (2000, 50, 10)
x_test.shape = (1000, 50, 1), y_test.shape = (1000, 50, 10)


In [3]:
class GatedActivationUnit(keras.layers.Layer):
    def __init__(self, activation='tanh', **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)

    def call(self, inputs, **kwargs):
        num_filters = inputs.shape[-1] // 2
        linear_output = self.activation(inputs[..., :num_filters])
        gate = keras.activations.sigmoid(inputs[..., num_filters:])
        return self.activation(linear_output) * gate


def wavenet_residual_block(inputs, num_filters, dilation_rate):
    x = keras.layers.Conv1D(2 * num_filters, kernel_size=2, padding='causal', dilation_rate=dilation_rate)(inputs)
    x = GatedActivationUnit()(x)
    x = keras.layers.Conv1D(num_filters, kernel_size=1)(x)
    return keras.layers.Add()([x, inputs]), x

In [4]:
utils.reset_session()

num_layers_per_block = 3  # 10 in the paper
num_blocks = 1            # 3 in the paper
num_filters = 32          # 128 in the paper
num_outputs = 10          # 256 in the paper

inputs = keras.layers.Input(shape=[None, 1])
x = keras.layers.Conv1D(num_filters, kernel_size=2, padding='causal')(inputs)
skip_to_last = []
for dilation_rate in [2**i for i in range(num_layers_per_block)] * num_blocks:
    x, skip = wavenet_residual_block(x, num_filters, dilation_rate)
    skip_to_last.append(skip)
x = keras.activations.relu(keras.layers.Add()(skip_to_last))
x = keras.layers.Conv1D(num_filters, kernel_size=1, activation='relu')(x)
outputs = keras.layers.Conv1D(num_outputs, kernel_size=1, activation='softmax')(x)

model = keras.models.Model(inputs=inputs, outputs=outputs)

def last_time_step_mse(y_true, y_pred):
    return keras.metrics.mean_squared_error(y_true[:, -1], y_pred[:, -1])

optimizer=keras.optimizers.Adam(lr=0.001)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', last_time_step_mse])
history = model.fit(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

Epoch 1/5
219/219 [==============================] - 2s 11ms/step - loss: 0.1300 - mae: 0.3058 - last_time_step_mse: 0.1260 - val_loss: 0.1229 - val_mae: 0.2985 - val_last_time_step_mse: 0.1199
Epoch 2/5
219/219 [==============================] - 2s 11ms/step - loss: 0.1222 - mae: 0.2979 - last_time_step_mse: 0.1178 - val_loss: 0.1217 - val_mae: 0.2977 - val_last_time_step_mse: 0.1189
Epoch 3/5
219/219 [==============================] - 3s 15ms/step - loss: 0.1215 - mae: 0.2976 - last_time_step_mse: 0.1172 - val_loss: 0.1214 - val_mae: 0.2979 - val_last_time_step_mse: 0.1186
Epoch 4/5
219/219 [==============================] - 3s 12ms/step - loss: 0.1212 - mae: 0.2975 - last_time_step_mse: 0.1169 - val_loss: 0.1211 - val_mae: 0.2974 - val_last_time_step_mse: 0.1183
Epoch 5/5
219/219 [==============================] - 3s 13ms/step - loss: 0.1210 - mae: 0.2973 - last_time_step_mse: 0.1167 - val_loss: 0.1209 - val_mae: 0.2973 - val_last_time_step_mse: 0.1181
